# Session Summary Dashboard

Quick stats for a single day or a complete multi-day run.

## Data generation

```bash
# Single week
./build/qrsdp_run --seed 42 --days 5 --seconds 23400

# Full year
./build/qrsdp_run --seed 42 --days 252 --seconds 23400
```

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go

import qrsdp_reader as reader
import book_replay as replay

In [2]:
# --- Configuration ---
RUN_DIR = Path("../output/run_42")

manifest = reader.load_manifest(RUN_DIR)
sessions = manifest["sessions"]
print(f"Run: {RUN_DIR.name}")
print(f"Seed: {manifest['base_seed']}, days: {len(sessions)}")
print(f"Date range: {sessions[0]['date']} — {sessions[-1]['date']}")

Run: run_42
Seed: 42, days: 5
Date range: 2026-01-02 — 2026-01-08


## Single-Day Summary

In [3]:
DAY_INDEX = 0

session = sessions[DAY_INDEX]
day_file = RUN_DIR / session["file"]
header = reader.read_header(day_file)
events = reader.read_day(day_file)

file_size_bytes = os.path.getsize(day_file)
raw_size = len(events) * 26
compression_ratio = raw_size / file_size_bytes if file_size_bytes > 0 else 0

duration_s = (events["ts_ns"][-1] - events["ts_ns"][0]) / 1e9 if len(events) > 1 else 0
event_rate = len(events) / duration_s if duration_s > 0 else 0

# Book replay for price stats
book = replay.replay_book(
    events,
    p0_ticks=header["p0_ticks"],
    levels_per_side=header["levels_per_side"],
    initial_spread_ticks=header["initial_spread_ticks"],
    initial_depth=header["initial_depth"],
)

shifts = (np.diff(book["best_bid"]) != 0) | (np.diff(book["best_ask"]) != 0)
n_shifts = shifts.sum()

print(f"\n{'='*50}")
print(f"  Date:             {session['date']}")
print(f"  Events:           {len(events):>12,}")
print(f"  Duration:         {duration_s:>12,.1f} s")
print(f"  Event rate:       {event_rate:>12,.0f} events/s")
print(f"  Open price:       {book['mid_ticks'][0]:>12.1f} ticks")
print(f"  Close price:      {book['mid_ticks'][-1]:>12.1f} ticks")
print(f"  Max spread:       {book['spread_ticks'].max():>12d} ticks")
print(f"  Price shifts:     {n_shifts:>12,}")
print(f"  File size:        {file_size_bytes / 1024 / 1024:>12.2f} MB")
print(f"  Raw size:         {raw_size / 1024 / 1024:>12.2f} MB")
print(f"  Compression:      {compression_ratio:>12.2f}x")
print(f"{'='*50}")


  Date:             2026-01-02
  Events:              2,262,506
  Duration:             23,400.0 s
  Event rate:                 97 events/s
  Open price:            10000.0 ticks
  Close price:            9781.0 ticks
  Max spread:                  2 ticks
  Price shifts:           40,680
  File size:               27.35 MB
  Raw size:                56.10 MB
  Compression:              2.05x


### Event Type Breakdown

In [4]:
type_counts = np.bincount(events["type"], minlength=6)
breakdown = pd.DataFrame({
    "Event Type": [reader.EVENT_TYPES.get(i, f"UNKNOWN_{i}") for i in range(6)],
    "Count": type_counts[:6],
    "Percentage": [f"{100 * c / len(events):.1f}%" for c in type_counts[:6]],
})
breakdown

,Event Type,Count,Percentage
0,ADD_BID,515055,22.8%
1,ADD_ASK,514506,22.7%
2,CANCEL_BID,170963,7.6%
3,CANCEL_ASK,170892,7.6%
4,EXECUTE_BUY,444760,19.7%
5,EXECUTE_SELL,446330,19.7%


## Multi-Day Summary Table

In [5]:
rows = []
for session_info in sessions:
    fpath = RUN_DIR / session_info["file"]
    hdr = reader.read_header(fpath)
    evts = reader.read_day(fpath)
    fsize = os.path.getsize(fpath)

    bk = replay.replay_book(
        evts,
        p0_ticks=hdr["p0_ticks"],
        levels_per_side=hdr["levels_per_side"],
        initial_spread_ticks=hdr["initial_spread_ticks"],
        initial_depth=hdr["initial_depth"],
    )

    dur = (evts["ts_ns"][-1] - evts["ts_ns"][0]) / 1e9 if len(evts) > 1 else 0
    sh = ((np.diff(bk["best_bid"]) != 0) | (np.diff(bk["best_ask"]) != 0)).sum()

    rows.append({
        "Date": session_info["date"],
        "Events": f"{len(evts):,}",
        "Rate (ev/s)": f"{len(evts) / dur:,.0f}" if dur > 0 else "—",
        "Open": f"{bk['mid_ticks'][0]:.1f}",
        "Close": f"{bk['mid_ticks'][-1]:.1f}",
        "Shifts": f"{sh:,}",
        "File MB": f"{fsize / 1024 / 1024:.2f}",
        "Compression": f"{len(evts) * 26 / fsize:.2f}x" if fsize > 0 else "—",
    })
    print(f"  {session_info['date']}: {len(evts):>10,} events, close={bk['mid_ticks'][-1]:.1f}")

summary_df = pd.DataFrame(rows)
summary_df

  2026-01-02:  2,262,506 events, close=9781.0
  2026-01-05:  2,260,762 events, close=9953.0
  2026-01-06:  2,261,351 events, close=9932.0
  2026-01-07:  2,263,572 events, close=9913.0
  2026-01-08:  2,262,111 events, close=9768.0


,Date,Events,Rate (ev/s),Open,Close,Shifts,File MB,Compression
0,2026-01-02,"2,262,506",97,10000.0,9781.0,"40,680",27.35,2.05x
1,2026-01-05,"2,260,762",97,9781.0,9953.0,"40,398",27.34,2.05x
2,2026-01-06,"2,261,351",97,9953.0,9932.0,"40,227",27.34,2.05x
3,2026-01-07,"2,263,572",97,9932.0,9913.0,"40,628",27.36,2.05x
4,2026-01-08,"2,262,111",97,9913.0,9768.0,"40,816",27.35,2.05x


### Closing Price Across Days

In [6]:
closes = [float(r["Close"]) for r in rows]
dates = [r["Date"] for r in rows]

fig_close = go.Figure(
    data=go.Scatter(
        x=dates, y=closes,
        mode="lines+markers", name="Close price",
        line=dict(color="#1976D2", width=2),
        marker=dict(size=6),
    ),
    layout=go.Layout(
        title="Daily Closing Mid-Price",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Price (ticks)"),
        height=400,
        template="plotly_white",
    ),
)
fig_close.show()